# Specifying Resource Request With GPU

## Requirements

- Authenticated to gcloud (```gcloud auth application-default login```)

This notebook demonstrate how to specify resource request such as CPU, memory, minimum/macimum number of replica, gpu name and gpu count for the model. Without specifying resource request the model will be deployed with system wide defaults. But, one model requires different resource compared to the others and specifying the proper resource will optimize the model performance.

In [ ]:
!pip install --upgrade -r requirements.txt > /dev/null

In [ ]:
import merlin
import warnings
import os
from sklearn import svm
from sklearn import datasets
from joblib import dump
from sklearn.datasets import load_iris
from merlin.model import ModelType
warnings.filterwarnings('ignore')

## 1. Initialize MLP Resources

### 1.1 Set MLP Server

In [ ]:
# Set MLP Server
merlin.set_url("localhost:8080/api/merlin")

### 1.2 Set Active Project

`project` represent a project in real life. You may have multiple model within a project.

`merlin.set_project(<project_name>)` will set the active project into the name matched by argument. You can only set it to an existing project. If you would like to create a new project, please do so from the MLP console at http://localhost:8080/projects/create.

In [ ]:
merlin.set_project("sample")

### 1.3 Set Active Model

`model` represents an abstract ML model. Conceptually, `model` in MLP is similar to a class in programming language. To instantiate a `model` you'll have to create a `model_version`.

Each `model` has a type, currently model type supported by MLP are: sklearn, xgboost, tensorflow, pytorch, and user defined model (i.e. pyfunc model).

`model_version` represents a snapshot of particular `model` iteration. You'll be able to attach information such as metrics and tag to a given `model_version` as well as deploy it as a model service.

`merlin.set_model(<model_name>, <model_type>)` will set the active model to the name given by parameter, if the model with given name is not found, a new model will be created.

In [ ]:
merlin.set_model("resource-request", ModelType.SKLEARN)

## 2. Train and Deploy

### 2.1 Train and Upload Model

`merlin.new_model_version()` is a convenient method to create a model version and start its development process. It is equal to following codes:

```
v = model.new_model_version()
v.start()
v.log_model(model_dir=model_dir)
v.finish()
```

In [ ]:
model_dir = "sklearn-model"
MODEL_FILE = "model.joblib"

url = ""

# Create new version of the model
with merlin.new_model_version() as v:
    clf = svm.SVC(gamma='scale')
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    clf.fit(X, y)
    dump(clf, os.path.join(model_dir, MODEL_FILE))
    
    # Upload the serialized model to MLP
    merlin.log_model(model_dir=model_dir)

## 2.2 Deploy Model With Resource Request With GPU

We will specify following resource request for the model which specifies `gpu_request` and `gpu_name` options in addition to previously known options.

In [ ]:
from merlin.resource_request import ResourceRequest

resource_req = ResourceRequest(min_replica=0, 
                               max_replica=2, 
                               cpu_request="1", 
                               memory_request="1Gi",
                               gpu_request="1",
                               gpu_name="nvidia-t4"
                               )

In [ ]:
endpoint = merlin.deploy(v, resource_request=resource_req)

## 2.3 Send Test Request

In [ ]:
%%bash -s "$endpoint.url"
curl -X POST $1 -d '{
  "instances": [
    [2.8,  1.0,  6.8,  0.4],
    [3.1,  1.4,  4.5,  1.6]
  ]
}'

## 3.4 Delete Deployment

In [ ]:
merlin.undeploy(v)